In [10]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdFMCS
import py3Dmol
import espsim
from espsim import EmbedAlignConstrainedScore,EmbedAlignScore

In [11]:
mols = [m for m in Chem.SDMolSupplier('icg001.sdf', removeHs=False)]

In [12]:
def drawTwoMols(m1,m2,p=None,confId1=-1,confId2=-1,viewergrid=(1,2)):
    if p is None:
        p = py3Dmol.view(width=800,height=400,viewergrid=viewergrid)
    p.removeAllModels(viewer=(0,0))
    p.removeAllModels(viewer=(0,1))
    mb1 = Chem.MolToMolBlock(m1,confId=confId1)
    p.addModel(mb1,'sdf',viewer=(0,0))
    mb2 = Chem.MolToMolBlock(m2,confId=confId2)
    p.addModel(mb2,'sdf',viewer=(0,1))
    p.setStyle({'stick':{'radius':0.15}},viewer=(0,0))
    p.setStyle({'stick':{'radius':0.15}},viewer=(0,1))
    p.zoomTo(viewer=(0,0))
    p.zoomTo(viewer=(0,1))
    return p

In [14]:
p = drawTwoMols(mols[0],mols[30])
p.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
mcs = rdFMCS.FindMCS(mols[:2],completeRingsOnly=True,atomCompare=rdFMCS.AtomCompare.CompareElements)

In [16]:
mcs.smartsString

'[#8]=&!@[#6]1-&@[#6](-&!@[#6](-&!@[#6]2:&@[#6](:&@[#6](:&@[#6](:&@[#6](:&@[#6]:&@2-&!@[#1])-&!@[#1])-&!@[#8]-&!@[#1])-&!@[#1])-&!@[#1])(-&!@[#1])-&!@[#1])(-&@[#7]2-&@[#6](-&@[#6](-&@[#6](-&@[#7](-&@[#6]-&@2(-&@[#6](-&@[#7]-&@1-&!@[#6](-&!@[#6]1:&@[#6](:&@[#6](:&@[#6](:&@[#6]2:&@[#6]:&@1:&@[#6](:&@[#6](:&@[#6](:&@[#6]:&@2-&!@[#1])-&!@[#1])-&!@[#1])-&!@[#1])-&!@[#1])-&!@[#1])-&!@[#1])(-&!@[#1])-&!@[#1])(-&!@[#1])-&!@[#1])-&!@[#1])-&!@[#6](-&!@[#7](-&!@[#6](-&!@[#6]1:&@[#6](:&@[#6](:&@[#6](:&@[#6](:&@[#6]:&@1-&!@[#1])-&!@[#1])-&!@[#1])-&!@[#1])-&!@[#1])(-&!@[#1])-&!@[#1])-&!@[#1])=&!@[#8])(-&!@[#1])-&!@[#1])(-&!@[#1])-&!@[#1])=&!@[#8])-&!@[#1]'

In [17]:
def getMCSmol(mol,mcs):
    mcsp = Chem.MolFromSmarts(mcs.smartsString)
    match = mol.GetSubstructMatch(mcsp)
    match_smi = Chem.MolFragmentToSmiles(mol,atomsToUse=match,canonical=True)
    return Chem.MolFromSmiles(match_smi)

In [ ]:
mcsmol = getMCSmol(mols[0],mcs)
AllChem.AddHs(mcsmol)
AllChem.EmbedMolecule(mcsmol,AllChem.ETKDGv2())
core = AllChem.DeleteSubstructs(AllChem.Replace)